In [8]:
import re, json, html
import pandas as pd
import numpy as np

def strip_tags(s: str) -> str:
    s = re.sub(r'<br\s*/?>', '\n', s, flags=re.I)
    s = re.sub(r'<[^>]+>', '', s)
    s = html.unescape(s)
    # normalize spaces
    s = re.sub(r'\xa0', ' ', s)  # non-breaking
    s = re.sub(r'[ \t]+', ' ', s).strip()
    return s

def to_number(x):
    x = x.strip()
    if x in {'.', '. .', ''}:
        return None
    # remove commas in thousands
    x = x.replace(',', '')
    try:
        if '.' in x:
            return float(x)
        return int(x) if x.isdigit() else x
    except ValueError:
        return x

with open("data/USCODE22_LLCP_102523.HTML", "r", encoding="utf-8", errors="ignore") as f:
    snippet = f.read()
    
table_blocks = re.findall(r'(<table class="table".*?</table>)', snippet, flags=re.S|re.I)


items = []
for tb in table_blocks:
    # header info
    m = re.search(r'<td[^>]*class="[^"]*linecontent[^"]*"[^>]*colspan="5"[^>]*>(.*?)</td>', tb, flags=re.S|re.I)
    meta = {}
    if m:
        meta_text = strip_tags(m.group(1))
        # Split on newlines and parse key: value
        for line in meta_text.split('\n'):
            if ':' in line:
                k, v = line.split(':', 1)
                meta[k.strip()] = v.strip()
    # body rows
    body = re.search(r'<tbody>(.*?)</tbody>', tb, flags=re.S|re.I)
    rows = []
    if body:
        for tr in re.findall(r'<tr>(.*?)</tr>', body.group(1), flags=re.S|re.I):
            tds = re.findall(r'<td[^>]*>(.*?)</td>', tr, flags=re.S|re.I)
            if len(tds) == 5:
                value = strip_tags(tds[0])
                vlabel = strip_tags(tds[1])
                freq = strip_tags(tds[2])
                perc = strip_tags(tds[3])
                wperc = strip_tags(tds[4])
                rows.append({
                    "value": value,
                    "label": vlabel,
                    "frequency": to_number(freq),
                    "percentage": to_number(perc),
                    "weighted_percentage": to_number(wperc),
                })
    if meta or rows:
        items.append({
            "sas_variable": meta.get("SAS Variable Name"),
            "label": meta.get("Label"),
            "section": meta.get("Section Name"),
            "module_number": meta.get("Module Number"),
            "question_number": meta.get("Question Number"),
            "column": meta.get("Column"),
            "type": meta.get("Type of Variable"),
            "question": meta.get("Question"),
            "raw_meta": meta,
            "categories": rows
        })

# Save to JSON file for download
json_path = "data/codebook_parsed.json"
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(items, f, ensure_ascii=False, indent=2)

# Build a convenient DataFrame for querying: one row per (variable, value)
records = []
for item in items:
    for c in item["categories"]:
        records.append({
            "sas_variable": item["sas_variable"],
            "variable_label": item["label"],
            "value": c["value"],
            "value_label": c["label"],
            "frequency": c["frequency"],
            "percentage": c["percentage"],
            "weighted_percentage": c["weighted_percentage"],
        })

df = pd.DataFrame.from_records(records)

json_path, len(items), df.shape


('data/codebook_parsed.json', 324, (2011, 7))

In [26]:
import json
data = pd.read_csv("data/train.csv")
with open("data/codebook_parsed.json") as f:
    dict = json.load(f)

In [136]:
cols = data.columns.tolist()

keep_cols = ["_RFHLTH", "_PHYS14D", "_MENT14D", "_HLTHPLN", "_HCVU652", "_TOTINDA", "_LTASTH1", "_CASTHM1", "_ASTHMS1",
             "_SEX", "_AGEG5YR", "_SMOKER3", "_CURECI2", "_PACKDAY", "_SMOKGRP", "_LCSREC", "_RFDRHV8", "_FLSHOT7","_PNEUMO3",
             "ID", "TARGET", '_STATE', 'LADULT1', 'COLGSEX1', 'LANDSEX1', 'CADULT1', 'CELLSEX1', 'SEXVAR',
                'GENHLTH', 'PHYSHLTH', 'MENTHLTH', 'POORHLTH', 'PRIMINSR', 'PERSDOC3', 'MEDCOST1',
                'CHECKUP1', 'EXERANY2', 'SLEPTIM1', 'CVDSTRK3', 'ASTHMA3',
                'ASTHNOW', 'CHCSCNC1', 'CHCOCNC1', 'CHCCOPD3', 'ADDEPEV3', 'CHCKDNY2', 'DIABETE4',
                'VETERAN3', 'WEIGHT2', 'HEIGHT3', 'DIFFWALK', 'DIFFDRES', 'DIFFALON', 'CERVSCRN',
                'CRVCLPAP', 'CIMEMLOS', 'CAREGIV1', 'SDHISOLT', 'SDHEMPLY', 'MARIJAN1', 'LASTSMK2',
                'ASBIDRNK', 'CRVCLHPV', 'HADHYST2', 'COLNSIGM', "COLNCNCR", 'VIRCOLO1', "SMALSTOL", 'STOOLDN2', 'SMOKE100', 'SMOKDAY2',
            'USENOW3', 'ECIGNOW2', 'LCSFIRST', 'LCSLAST', 'LCSNUMCG', 'LCSCTSC1', 'LCSSCNCR', 'ALCDAY4', 'AVEDRNK3',
            'DRNK3GE5', "MAXDRNKS", 'HIVRISK5', 'COVIDPOS', 'COVIDSMP', 'COVIDPRM', 'PDIABTS1', 'PREDIAB2', 'DIABTYPE',
            'FEETSORE', 'TOLDCFS', 'HAVECFS', 'COPDCOGH', 'COPDFLEM', 'COPDBRTH', 'COPDBTST', 'COPDSMOK', 'CNCRDIFF', 
            'CSRVDOC1', 'CSRVDEIN', 'CSRVPAIN', 'CSRVCTL2', 'PSATEST1', 'PCPSARS2', 'COLNTES1', 'HADSIGM4', "SIGMTES1", 'LASTSIG4', 'VCLNTES2', 'STOLTEST', 'SDNATES1', 'LCSCTWHN', 'BLDSTFIT', 
             'FLUSHOT7', 'FLSHTMY3', 'PNEUVAC4', 'TETANUS1', 'HIVTST7', 'HIVTSTD3', 'IMFVPLA3', 'HPVADVC4', 'HPVADSHT',
             'SHINGLE2', 'COVIDVA1', 'COVIDNU1', 'COVIDFS1', 'COVIDSE1', 'PSATIME1'] 

remove_cols = ["QSTLANG", "_METSTAT","_URBSTAT", "MSCODE", "_STSTR", "_STRWT", "_RAWRAKE", "_WT2RAKE", "_MRACE2", "_IMPRACE",
             "_IMPRACE","_CHISPNC", "_CRACE2", "_CPRACE2", "CAGEG", "_CLLCPWT", "_DUALUSE","_PRACE2", "_HISPANC","_RACE1",
             "_DUALCOR", "_LLCPWT", "_LLCPWT2", "_EXTETH3", "_ALTETH3", "_DENVST3", "_DRDXAR2", "_RACEG22", "_RACEGR4", "_RACEPR1",
             "_AGE65YR", "_AGE80", "_AGE_G", "HTIN4", "HTM4","WTKG3", "WTKG3", "_BMI5", "_RFBMI5","_BMI5CAT", "_CHLDCNT", "_EDUCAG",
             "_INCOMG1", "_RFMAM22", "_MAM5023", "_HADCOLN", "_CLNSCP1","_HADSIGM", "_SGMSCP1", "_SGMS101", "_RFBLDS5", "_STOLDN1",
             "_VIRCOL1", "_SBONTI1", "_CRCREC2", "_RFSMOK3", "_YRSSMOK", "_PACKYRS", "_YRSQUIT", "DRNKANY6", "DROCDY4_", "_RFBING6",
             "_DRNKWK2", "_AIDTST4", 'FMONTH', 'IDATE', 'IMONTH', 'IDAY', 'IYEAR', 'DISPCODE', 'SEQNO', '_PSU', 'CTELENM1',
                  'PVTRESD1', 'COLGHOUS', 'STATERE1', 'CELPHON1', 'NUMADULT', 'NUMMEN', 'NUMWOMEN',
                  'RESPSLCT', 'SAFETIME', 'CTELNUM1', 'CELLFON5', 'PVTRESD3', 'CCLGHOUS', 'CSTATE1',
                  'LANDLINE', 'HHADULT', 'LASTDEN4', 'RMVTETH4', 'HAVARTH4', 'MARITAL', 'EDUCA',
                  'RENTHOM1', 'NUMHHOL4', 'EMPLOY1', 'CHILDREN', 'INCOME3', 'PREGNANT', 'DEAF', 'BLIND',
                  'DECIDE', 'HADMAM', 'HOWLONG', 'CRVCLCNC', 'PCSTALK1', 'CDHOUSE', 'CDASSIST', 'CDHELP',
                  'CDSOCIAL', 'CDDISCUS', 'CRGVREL4', 'CRGVLNG1', 'CRGVHRS1', 'CRGVPRB3', 'CRGVALZD',
                  'CRGVPER1', 'CRGVHOU1', 'CRGVEXPT', 'ACEDEPRS', 'ACEDRINK', 'ACEDRUGS', 'ACEPRISN',
                  'ACEDIVRC', 'ACEPUNCH', 'ACEHURT1', 'ACESWEAR', 'ACETOUCH', 'ACETTHEM', 'ACEHVSEX',
                  'ACEADSAF', 'ACEADNED', 'EMPSUPRT', 'LSATISFY', 'FOODSTMP', 'SDHFOOD1', 'SDHBILLS',
                  'SDHUTILS', 'SDHTRNSP', 'SDHSTRE1', 'MARJSMOK', 'MARJEAT', 'MARJVAPE', 'MARJDAB',
                  'MARJOTHR', 'STOPSMK2', 'MENTCIGS', 'MENTECIG', 'HEATTBCO', 'ASBIALCH', 'ASBIBING',
                  'ASBIADVC', 'ASBIRDUC', 'FIREARM5', 'GUNLOAD', 'LOADULK2', 'RCSGEND1', 'RCSXBRTH',
                  'RCSRLTN2', 'CASTHDX2', 'CASTHNO2', 'BIRTHSEX', 'SOMALE', 'SOFEMALE', 'TRNSGNDR',
                  'HADSEX', 'PFPPRVN4', 'TYPCNTR9', 'BRTHCNT4', 'WHEREGET', 'NOBCUSE8', 'BCPREFER',
                  'RRCLASS3', 'RRCOGNT2', 'RRTREAT', 'RRATWRK2', 'RRHCARE4', 'RRPHYSM2' , 'INSULIN1', 'CHKHEMO3', 'EYEEXAM1', 'DIABEYE1', 'DIABEDU1', 'WORKCFS', 'CSRVSUM', 'CSRVRTRN', 'CSRVINST',
             'CSRVINSR', 'CSRVCLIN', 'PSASUGST', 'CVDINFR4', 'CVDCRHD4', "DIABAGE4","NUMPHON4","CPDEMO1C", "CNCRAGE","CNCRTYP2", "CSRVTRT3"]



tbd_cols = []
blank_list = []

for col in cols:
    if col not in keep_cols and col not in remove_cols and col not in tbd_cols:
        print(f"Variable: {col}")
        entry = next((item for item in dict if item["sas_variable"] == col), None)
        if entry:
            total = 0
            print(f"Label: {entry['label']}")
            print(f"Question: {entry['question']}")
            print("Categories:")
            for cat in entry["categories"]:
                print(f"  Value: {cat['value']}, Label: {cat['label']}, Freq: {cat['frequency']}, Perc: {cat['percentage']}")
                total += cat['frequency'] if cat['frequency'] is not None else 0

            if "BLANK" in [cat['value'] for cat in entry["categories"]]:
                blank_cat = next(cat for cat in entry["categories"] if cat['value'] == "BLANK")
                percent_blank = blank_cat["percentage"]

                if percent_blank is not None and percent_blank > 80:
                    blank_list.append(col)
                    print(f"  --> Added to blank_list (percent blank: {percent_blank}%)")
        else:
            print("No dictionary entry found.")
        print("\n")




Variable: EMTSUPRT
Label: How often get emotional support needed
Question: How often do you get the social and emotional support you need?
Categories:
  Value: 1, Label: Always, Freq: 118012, Perc: 46.44
  Value: 2, Label: Usually, Freq: 77907, Perc: 30.66
  Value: 3, Label: Sometimes, Freq: 33813, Perc: 13.3
  Value: 4, Label: Rarely, Freq: 10835, Perc: 4.26
  Value: 5, Label: Never, Freq: 9379, Perc: 3.69
  Value: 7, Label: Dont know/Not sure, Freq: 2820, Perc: 1.11
  Value: 9, Label: Refused, Freq: 1375, Perc: 0.54
  Value: BLANK, Label: Not asked or Missing, Freq: 190991, Perc: None


Variable: USEMRJN4
No dictionary entry found.


Variable: QSTVER
Label: Questionnaire Version Identifier
Question: Questionnaire Version Identifier
Categories:
  Value: 10, Label: Landline single questionnaire, Freq: 60413, Perc: 13.57
  Value: 11, Label: Landline multiple questionnaire version 1, Freq: 16065, Perc: 3.61
  Value: 12, Label: Landline multiple questionnaire version 2, Freq: 14921, Perc:

In [ ]:
import yaml

with open("features.yaml","r") as f:
    features = yaml.load(f, Loader=yaml.FullLoader)

data[features['features']]

,_STATE,LADULT1,COLGSEX1,LANDSEX1,CADULT1,CELLSEX1,SEXVAR,GENHLTH,PHYSHLTH,MENTHLTH,...,_SMOKER3,_CURECI2,_PACKDAY,_SMOKGRP,_LCSREC,_RFDRHV8,_FLSHOT7,_PNEUMO3,ID,TARGET
0,2.0,1.0,NaN,2.0,NaN,NaN,2.0,3.0,30.0,88.0,...,3.0,1.0,2.00,3.0,NaN,1.0,1.0,1.0,0,True
1,9.0,NaN,NaN,NaN,1.0,2.0,2.0,3.0,88.0,15.0,...,4.0,1.0,NaN,4.0,NaN,9.0,NaN,NaN,1,False
2,19.0,1.0,NaN,NaN,NaN,NaN,1.0,2.0,88.0,88.0,...,3.0,1.0,0.75,3.0,NaN,1.0,NaN,NaN,2,False
3,39.0,NaN,NaN,NaN,1.0,1.0,1.0,3.0,88.0,88.0,...,4.0,1.0,NaN,4.0,NaN,1.0,NaN,NaN,3,False
4,25.0,NaN,NaN,NaN,1.0,1.0,1.0,3.0,88.0,3.0,...,4.0,1.0,NaN,4.0,NaN,1.0,NaN,NaN,4,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224995,2.0,NaN,NaN,NaN,1.0,1.0,1.0,3.0,88.0,88.0,...,3.0,1.0,1.00,2.0,2.0,1.0,2.0,9.0,224995,True
224996,17.0,NaN,NaN,NaN,1.0,1.0,1.0,3.0,15.0,15.0,...,3.0,1.0,0.75,3.0,NaN,1.0,NaN,NaN,224996,False
224997,38.0,NaN,NaN,NaN,1.0,1.0,1.0,3.0,88.0,88.0,...,4.0,1.0,NaN,4.0,NaN,1.0,NaN,NaN,224997,False
224998,25.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,10.0,88.0,...,4.0,1.0,NaN,4.0,NaN,1.0,NaN,NaN,224998,False


DATA CLEAN